In [1]:
from torch import nn
import torch
import numpy as np
import pandas as pd
import pickle, time
import re, os, string, typing, gc, json
import torch.nn.functional as F
import spacy
from sklearn.model_selection import train_test_split
from collections import Counter
import spacy
from spacy.cli.download import download
download(model="en_core_web_sm")
from preprocess import *
%load_ext autoreload
%autoreload 2

nlp = spacy.load('en_core_web_sm')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
def parse_data(data:dict)->list:
    '''
    Parses the JSON file of Squad dataset by looping through the
    keys and values and returns a list of dictionaries with
    context, query and label triplets being the keys of each dict.
    '''
    data = data['data']
    qa_list = []

    for paragraphs in data:

        for para in paragraphs['paragraphs']:
            context = para['context']

            for qa in para['qas']:
                
                id = qa['id']
                question = qa['question']
                
                for ans in qa['answers']:
                    answer = ans['text']
                    ans_start = ans['answer_start']
                    ans_end = ans_start + len(answer)
                    
                    qa_dict = {}
                    qa_dict['id'] = id
                    qa_dict['context'] = context
                    qa_dict['question'] = question
                    qa_dict['label'] = [ans_start, ans_end]

                    qa_dict['answer'] = answer
                    qa_list.append(qa_dict)    

    
    return qa_list

In [3]:
# load dataset json files

with open("train.json") as file:
    train_data=json.loads(file.read())
    
with open("dev.json") as file1:
    valid_data=json.loads(file1.read())

# parse the json structure to return the data as a list of dictionaries

train_list = parse_data(train_data)
valid_list = parse_data(valid_data)
print('--------------------------')

print('Train list len: ',len(train_list))
print('Valid list len: ',len(valid_list))

# converting the lists into dataframes

train_df = pd.DataFrame(train_list)[:400]
valid_df = pd.DataFrame(valid_list)[:400]

--------------------------
Train list len:  86821
Valid list len:  20302


In [4]:
train_df.head()

id  \
0  56be85543aeaaa14008c9063   
1  56be85543aeaaa14008c9065   
2  56be85543aeaaa14008c9066   
3  56bf6b0f3aeaaa14008c9601   
4  56bf6b0f3aeaaa14008c9602   

                                             context  \
0  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
1  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
2  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
3  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
4  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   

                                            question       label  \
0           When did Beyonce start becoming popular?  [269, 286]   
1  What areas did Beyonce compete in when she was...  [207, 226]   
2  When did Beyonce leave Destiny's Child and bec...  [526, 530]   
3      In what city and state did Beyonce  grow up?   [166, 180]   
4         In which decade did Beyonce become famous?  [276, 286]   

                answer  
0    in the late 1990s  
1  singing and dancing  
2                 2003  
3       Houston, Texas  
4           late 1990s

In [5]:
def preprocess_df(df):
    
    def to_lower(text):
        return text.lower()

    df.context = df.context.apply(to_lower)
    df.question = df.question.apply(to_lower)
    df.answer = df.answer.apply(to_lower)

In [6]:
preprocess_df(train_df)
preprocess_df(valid_df)

In [7]:
def gather_text_for_vocab(dfs:list):
    '''
    Gathers text from contexts and questions to build a vocabulary.
    
    :param dfs: list of dataframes of SQUAD dataset.
    :returns: list of contexts and questions
    '''
    
    text = []
    total = 0
    for df in dfs:
        unique_contexts = list(df.context.unique())
        unique_questions = list(df.question.unique())
        total += df.context.nunique() + df.question.nunique()
        text.extend(unique_contexts + unique_questions)
    
    assert len(text) == total
    
    return text

In [8]:
# gather text to build vocabularies

%time vocab_text = gather_text_for_vocab([train_df, valid_df])
print("Number of sentences in dataset: ", len(vocab_text))

CPU times: total: 15.6 ms
Wall time: 15.6 ms
Number of sentences in dataset:  613


In [9]:
def build_word_vocab(vocab_text):
    '''
    Builds a word-level vocabulary from the given text.
    
    :param list vocab_text: list of contexts and questions
    :returns 
        dict word2idx: word to index mapping of words
        dict idx2word: integer to word mapping
        list word_vocab: list of words sorted by frequency
    '''
    
    
    words = []
    for sent in vocab_text:
        for word in nlp(sent, disable=['parser','tagger','ner']):
            words.append(word.text)

    word_counter = Counter(words)
    word_vocab = sorted(word_counter, key=word_counter.get, reverse=True)
    print(f"raw-vocab: {len(word_vocab)}")
    word_vocab.insert(0, '<unk>')
    word_vocab.insert(1, '<pad>')
    print(f"vocab-length: {len(word_vocab)}")
    word2idx = {word:idx for idx, word in enumerate(word_vocab)}
    print(f"word2idx-length: {len(word2idx)}")
    idx2word = {v:k for k,v in word2idx.items()}
    
    
    return word2idx, idx2word, word_vocab

def build_char_vocab(vocab_text):
    '''
    Builds a character-level vocabulary from the given text.
    
    :param list vocab_text: list of contexts and questions
    :returns 
        dict char2idx: character to index mapping of words
        list char_vocab: list of characters sorted by frequency
    '''
    
    chars = []
    for sent in vocab_text:
        for ch in sent:
            chars.append(ch)

    char_counter = Counter(chars)
    char_vocab = sorted(char_counter, key=char_counter.get, reverse=True)
    print(f"raw-char-vocab: {len(char_vocab)}")
    high_freq_char = [char for char, count in char_counter.items() if count>=20]
    char_vocab = list(set(char_vocab).intersection(set(high_freq_char)))
    print(f"char-vocab-intersect: {len(char_vocab)}")
    char_vocab.insert(0,'<unk>')
    char_vocab.insert(1,'<pad>')
    char2idx = {char:idx for idx, char in enumerate(char_vocab)}
    print(f"char2idx-length: {len(char2idx)}")
    
    return char2idx, char_vocab

In [10]:
# build word and character-level vocabularies

%time word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)

C:\Users\yashs\anaconda3\envs\pyproject\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


raw-vocab: 2909
vocab-length: 2911
word2idx-length: 2911
CPU times: total: 1.52 s
Wall time: 2.1 s


In [11]:
print("----------------------------------")
%time char2idx, char_vocab = build_char_vocab(vocab_text)

----------------------------------
raw-char-vocab: 69
char-vocab-intersect: 48
char2idx-length: 50
CPU times: total: 0 ns
Wall time: 8.97 ms


In [12]:
def context_to_ids(text, word2idx):
    '''
    Converts context text to their respective ids by mapping each word
    using word2idx. Input text is tokenized using spacy tokenizer first.
    
    :param str text: context text to be converted
    :returns list context_ids: list of mapped ids
    
    :raises assertion error: sanity check
    
    '''

    context_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids
    
def question_to_ids(text, word2idx):
    '''
    Converts question text to their respective ids by mapping each word
    using word2idx. Input text is tokenized using spacy tokenizer first.
    
    :param str text: question text to be converted
    :returns list context_ids: list of mapped ids
    
    :raises assertion error: sanity check
    
    '''

    question_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    question_ids = [word2idx[word] for word in question_tokens]
    
    assert len(question_ids) == len(question_tokens)
    return question_ids

In [13]:
# # numericalize context and questions for training and validation set

# %time train_df['context_ids'] = train_df.context.apply(context_to_ids, word2idx=word2idx)
# %time valid_df['context_ids'] = valid_df.context.apply(context_to_ids, word2idx=word2idx)
# %time train_df['question_ids'] = train_df.question.apply(question_to_ids, word2idx=word2idx)
# %time valid_df['question_ids'] = valid_df.question.apply(question_to_ids, word2idx=word2idx)

In [14]:
def get_error_indices(df, idx2word):
    '''
    Gets error indices from the method above and returns a 
    set of those indices.
    '''
    
    start_value_error, end_value_error, assert_error = test_indices(df, idx2word)
    err_idx = start_value_error + end_value_error + assert_error
    err_idx = set(err_idx)
    print(f"Error indices: {len(err_idx)}")
    
    return err_idx

def test_indices(df, idx2word):
    start_value_error = []
    end_value_error = []
    assert_error = []
    for index, row in df.iterrows():

        answer_tokens = [w.text for w in nlp(row['answer'], disable=['parser','tagger','ner'])]

        start_token = answer_tokens[0]
        end_token = answer_tokens[-1]
        
        context_span  = [(word.idx, word.idx + len(word.text)) 
                         for word in nlp(row['context'], disable=['parser','tagger','ner'])]

        starts, ends = zip(*context_span)

        answer_start, answer_end = row['label']

        try:
            start_idx = starts.index(answer_start)
        except Exception as e:    
            print(e)
            start_value_error.append(index)
        try:
            end_idx  = ends.index(answer_end)
        except Exception as e:    
            print(e)
            end_value_error.append(index)

        try:
            assert idx2word[row['context_ids'][start_idx]] == answer_tokens[0]
            assert idx2word[row['context_ids'][end_idx]] == answer_tokens[-1]
        except Exception as e:    
            print(e)
            assert_error.append(index)


    return start_value_error, end_value_error, assert_error

In [15]:
train_df.shape

(400, 5)

In [16]:
# get indices with tokenization errors and drop those indices 

train_err = get_error_indices(train_df, idx2word)
valid_err = get_error_indices(valid_df, idx2word)

train_df.drop(train_err, inplace=True)
valid_df.drop(valid_err, inplace=True)

'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'conte

'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'context_ids'
'conte

In [17]:
def index_answer(row, idx2word):
    '''
    Takes in a row of the dataframe or one training example and
    returns a tuple of start and end positions of answer by calculating 
    spans.
    '''
    
    context_span = [(word.idx, word.idx + len(word.text)) for word in nlp(row[0], disable=['parser','tagger','ner'])]
    starts, ends = zip(*context_span)
    
    answer_start, answer_end = row[1]
    start_idx = starts.index(answer_start)
 
    end_idx  = ends.index(answer_end)
    
    ans_toks = [w.text for w in nlp(row[3],disable=['parser','tagger','ner'])]
    ans_start = ans_toks[0]
    ans_end = ans_toks[-1]
    assert idx2word[row.context_ids[start_idx]] == ans_start
    assert idx2word[row.context_ids[end_idx]] == ans_end
    
    return [start_idx, end_idx]

In [18]:
# get start and end positions of answers from the context
# this is basically the label for training QA models

train_label_idx = train_df.apply(index_answer(vocab_text,idx2word), axis=1, idx2word=idx2word)
valid_label_idx = valid_df.apply(index_answer(vocab_text,idx2word), axis=1, idx2word=idx2word)

train_df['label_idx'] = train_label_idx
valid_df['label_idx'] = valid_label_idx

ValueError: too many values to unpack (expected 2)

In [19]:
vocab_text

['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r&b girl-group destiny\'s child. managed by her father, mathew knowles, the group became one of the world\'s best-selling girl groups of all time. their hiatus saw the release of beyoncé\'s debut album, dangerously in love (2003), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number-one singles "crazy in love" and "baby boy".',
 'following the disbandment of destiny\'s child in june 2005, she released her second solo album, b\'day (2006), which contained hits "déjà vu", "irreplaceable", and "beautiful liar". beyoncé also ventured into acting, with a golden globe-nominated performance in dreamgirls (2006), and starring ro